In [0]:
import delta

In [0]:
# Esta camada de código tem o objetivo de fazer a ingestão de um arquivo Full Load na pasta de Bronze. O arquivo Full Load é uma Tabela em formato CSV, que contém dados mas que precisam ser atualizados através de arquivos CDC (Change Data Capture).

# Nesta parte do código, precisamos primeiro ler o arquivo CSV com o Spark que logo após a leitura se transforma em um Dataframe. No código podemos notar .option("header", "true"), utilizei esse comando pois com ele conseguimos nos comunicar com o Spark e dizer que o arquivo CSV contém cabeçalho, ou seja, nome de colunas presentes na primeira.
df_full_table = spark.read.format("csv").option("header", "true").load("/Volumes/raw/olist_ecommerce/full_load/customers/")

# Logo após a leitura preciso salvar o Dataframe no Schema de Bronze.
(df_full_table.coalesce(1)    # Com coalesce(1), garantimos que o Spark nos entregue apenas 1 arquivo salvo.
  .write
  .format("delta")    # Formato de salvamento do arquivo.
  .mode("overwrite")    # Modo de salvamento, aqui se caso ja contesse dados no Schema o "overwrite" iria subscreve-los.
  .saveAsTable("bronze.olist_ecommerce.customers"))   # Aqui é onde dizemos para o Spark o caminho de salvamento, e especicamos que no Schema o arquivo será salvo como Tabela.

In [0]:
%sql
SELECT * FROM bronze.olist_ecommerce.customers
LIMIT 100

-- Apenas verifico aqui se o arquivo foi salvo como Tabela com SQL na pasta de Bronze.

In [0]:
# Nesta camada de código preciso da importação de arquivos CDC. Onde no futuro esses arquivos que precisam ser atualizados serão a principal fonte de dados para a atualização da Tabela presente no Schema Bronze. 

# Importando e fazendo a leitura de dados com o Spark.
(spark.read
    .format("csv")
    .option("header", "true")
    .load("/Volumes/raw/olist_ecommerce/cdc/customers/")
    .createOrReplaceTempView("customers"))  # Essa linha de código nos permite transformar o Dataframe que foi lido pelo Spark em uma Tabela Temporária, nos permitindo fazer consultas SQL.

query = """ 
SELECT *
FROM customers
QUALIFY ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY modifed_at DESC) = 1
"""     # Nesta parte do código estamos fazendo uma consulta SQL, onde tenho o objetivo de fazer um filtro com que apareça os dados mais atualizados de cada cliente presente no arquivo CDC.

df_cdc_table = spark.sql(query)     # Transformando o resultado da consulta SQL em um Dataframe.
df_cdc_table.display()

In [0]:
# Aqui é onde iremos fazer um Merge para atualizar a Tabela presente no Schema Bronze. A Tabela só contém dados Full Load desatualizados, e com os dados obtidos acima através de arquivos CDC, iremos fazer um Merge para a atualização da Tabela.

bronze = delta.DeltaTable.forName(spark, "bronze.olist_ecommerce.customers") # Aqui conseguimos fazer com que, conversamos com nossa tabela através de uma variável.

(bronze.alias("b")
     .merge(df_cdc_table.alias("c"), "b.customer_id = c.customer_id")    # Fazendo um JOIN entre as tabelas.
     .whenMatchedDelete(condition = "c.OP = 'D'")      # Condição para deletar um dado presente na Tabela Bronze.
     .whenMatchedUpdateAll(condition = "c.OP = 'U'")   # Condição para atualizar um dado presente na Tabela Bronze
     .whenNotMatchedInsertAll(condition = "c.OP = 'I' OR c.OP = 'U'") # Condição para inserir um dado novo na Tabela Bronze.
     .execute()

)

In [0]:
%sql
SELECT * FROM bronze.olist_ecommerce.customers
WHERE customer_id = '00a16acd591b4bb112f2e94c7d2f9936'

